In [3]:
import sys
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.job import Job
from awsglue.dynamicframe import DynamicFrame
import re
from pyspark.sql.functions import col, when, last_day, to_date, sum as _sum
from pyspark.sql import functions as F

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
# Simula los argumentos como si los estuvieras pasando por CLI
sys.argv = [
    'glue_script.py',  # nombre ficticio del script, solo para cumplir con la forma
    '--JOB_NAME', 'cartera_2r',
    '--PAIS', '2',
    '--ANO', '2025',
    '--MES', '1'
]

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
# Obtener argumentos del job
args = getResolvedOptions(sys.argv, ['JOB_NAME', 'PAIS', 'ANO', 'MES'])

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
# Inicializar GlueContext y Job
sc = SparkContext.getOrCreate()
glueContext = GlueContext(sc)
spark = glueContext.spark_session
job = Job(glueContext)
job.init(args['JOB_NAME'], args)

logger = glueContext.get_logger()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
def select_table_from_rd(table):
    node = glueContext.create_dynamic_frame.from_options(
    connection_type="redshift",
    connection_options={
        "redshiftTmpDir": "s3://aws-glue-assets-575238426438-us-east-1/temporary/",
        "useConnectionProperties": "true",
        "dbtable": table,
        "connectionName": "conexion-redshift"
    },
    transformation_ctx="node"
    )
    return node

def select_query_from_rd(query):
    node = glueContext.create_dynamic_frame.from_options(
        connection_type="redshift",
        connection_options={
            "sampleQuery": query,
            "redshiftTmpDir": "s3://aws-glue-assets-575238426438-us-east-1/temporary/",
            "useConnectionProperties": "true",
            "connectionName": "conexion-redshift"
        },
        transformation_ctx="node"
    )
    return node


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
pais = args['PAIS'].zfill(2)
ano = args['ANO']
mes = args['MES'].zfill(2)
esquema_pais = {
    '01': 'silver',
    '02': 'silver_pan'
}
empresa_pais = {
    '01': 'JA',
    '02': 'JP'
}

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# QUERYS

In [16]:
query_cartera = f"""
--consulta cartera--
with tasas_wac as(
	SELECT
	last_day(to_date(v.campo1, 'd/mm/yyyy')) unidad2,
	v.campo2 WAC_NM
	  FROM bronze_col.variablesrentabilidadexcel v, silver.tipos_excel_variables t
	  WHERE t.id = v.excelid
	  AND t.nombre='COSTO_FINANCIERO_WACC'
),
param AS (
  SELECT
    valor,
    grupo,
    periodo
  FROM bronze_col.parametros_r
)
select
	c.CART_UNIDAD,
	c.CART_EMISION,
	c.CART_TRAMO_NUEVO,
	c.CART_RECAUDO_CAPITAL,
	c.CART_TIPOCUENTA,
	c.CART_CAPITAL,
	c.CART_TIPCRE,
	c.CART_CLIENTE,
	c.CART_CTACAR,
	c.CART_AGE_ULT_COMPRA,
	c.CART_PROV_LISIM_CAPITAL_INI,
	c.CART_SALPROV,
	c.CART_PROVISION,
	c.CART_SALDOPROVINI,
	c.CART_RECPER,
	c.CART_DCTO_CAPITAL,
	c.CART_SALPROVCAPITAL,
	c.CART_TRAMOINI,
	c.CART_RECAUDO_HONORARIOS,
	c.CART_ESTADO,
	c.CART_ESTINI,
	c.CART_SEGMENTO,
	c.CART_PRIMERA_ACC_SUGERIDA,
	c.CART_MARCADATA,
	c.CART_PRIMERA_MARCADATA,
	c.PROVISION_CAPITAL_NUEVO,
	c.CART_ACCIONSUGERIDA,
	c.CART_SALDO,
	c.CART_SALDO_CAPITAL_INI,
	c.CART_RECAUDO_INTERESMORA,
	c.CART_RECAUDO_GASTOSCOBRANZA,
	c.PROV_NUEVA_AJUSTADA,
	c.CART_RECAUDO_INTERESFINANCIERO,
	c.CART_CUOTASFAC,
	c.CART_PILOTO, 								------------ muchos campos
	LAST_DAY(TO_DATE(CART_UNIDAD, 'DDMMYYYY')) as UNIDAD2,
	LAST_DAY(CART_EMISION) as PERIODO,
	case
		when c.cart_tramo_nuevo is not null
			and c.cart_tramo_nuevo in (select valor from param where grupo = 'CART_TRAMO_{pais}') then c.cart_recaudo_capital
		else 0
	end as DEUDAS_MALAS,
	case
		when c.cart_tipocuenta not in (Select valor from param where grupo = 'CART_TCUENTA_{pais}')
			and c.cart_tramo_nuevo is not null
			and c.cart_tramo_nuevo in (select valor from param where grupo = 'CART_TRAMO_{pais}') then 0
	    when c.CART_TIPOCUENTA not in (Select valor from param where grupo = 'CART_TCUENTA_{pais}')
	        then cast((select valor from param where grupo = 'CFONDEO_{pais}' and periodo = '2024') as float) * c.CART_CAPITAL  		----------------------Ha estado en 0 desde 2024 --cambiar por AÑO
    else null
	end as COSTO_FONDEO,
	case
		when u.unidad_fecha >= to_date('20220101', 'yyyymmdd') 				
			and c.cart_tipocuenta not in (Select valor from param where grupo = 'CART_TCUENTA_{pais}')
			and c.cart_tramo_nuevo is not null
			and c.cart_tramo_nuevo in (select valor from param where grupo = 'CART_TRAMO_{pais}') then 0
		when u.unidad_fecha >= to_date('20220101', 'yyyymmdd')
			and c.cart_tipocuenta not in (Select valor from param where grupo = 'CART_TCUENTA_{pais}')
			then cast(tw.wac_nm as float) * c.cart_capital
		else 0
	end as COSTO_FONDEO_WACC,
	ROW_NUMBER() OVER (PARTITION BY c.CART_CLIENTE, c.CART_CTACAR, LAST_DAY(c.CART_EMISION)
                   ORDER BY LAST_DAY(TO_DATE(c.CART_UNIDAD, 'DDMMYYYY'))) AS meses_maduracion,
    '{empresa_pais.get(pais)}' as CART_EMP
from
	{esquema_pais.get(pais)}.CARTERA c,
	gold.unidad_tiempo u,
	tasas_wac tw
where
	c.CART_TIPCRE in (select valor from param where grupo = 'CART_TCRE_{pais}')
	and c.cart_unidad = u.unidad_codigo
	and u.unidad_fecha >= to_date('{ano}{mes}01', 'yyyymmdd')	
	and u.unidad_fecha <= last_day(to_date('{ano}{mes}01', 'yyyymmdd'))	
	and LAST_DAY(TO_DATE(c.CART_UNIDAD, 'DDMMYYYY')) = tw.unidad2
"""

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:
print(query_cartera)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…


--consulta cartera--
with tasas_wac as(
	SELECT
	last_day(to_date(v.campo1, 'd/mm/yyyy')) unidad2,
	v.campo2 WAC_NM
	  FROM bronze_col.variablesrentabilidadexcel v, silver.tipos_excel_variables t
	  WHERE t.id = v.excelid
	  AND t.nombre='COSTO_FINANCIERO_WACC'
),
param AS (
  SELECT
    valor,
    grupo,
    periodo
  FROM bronze_col.parametros_r
)
select
	c.CART_UNIDAD,
	c.CART_EMISION,
	c.CART_TRAMO_NUEVO,
	c.CART_RECAUDO_CAPITAL,
	c.CART_TIPOCUENTA,
	c.CART_CAPITAL,
	c.CART_TIPCRE,
	c.CART_CLIENTE,
	c.CART_CTACAR,
	c.CART_AGE_ULT_COMPRA,
	c.CART_PROV_LISIM_CAPITAL_INI,
	c.CART_SALPROV,
	c.CART_PROVISION,
	c.CART_SALDOPROVINI,
	c.CART_RECPER,
	c.CART_DCTO_CAPITAL,
	c.CART_SALPROVCAPITAL,
	c.CART_TRAMOINI,
	c.CART_RECAUDO_HONORARIOS,
	c.CART_ESTADO,
	c.CART_ESTINI,
	c.CART_SEGMENTO,
	c.CART_PRIMERA_ACC_SUGERIDA,
	c.CART_MARCADATA,
	c.CART_PRIMERA_MARCADATA,
	c.PROVISION_CAPITAL_NUEVO,
	c.CART_ACCIONSUGERIDA,
	c.CART_SALDO,
	c.CART_SALDO_CAPITAL_INI,
	c.CART_RECAUDO_INTERESMORA,
	c.C

In [18]:
cartera_node = select_query_from_rd(query_cartera) 
cartera_df = cartera_node.toDF()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

/home/glue_user/spark/python/pyspark/sql/dataframe.py:127: UserWarning: DataFrame constructor is internal. Do not directly use it.
  warnings.warn("DataFrame constructor is internal. Do not directly use it.")

In [ ]:
cartera_df.show()

In [20]:
cartera_node_w = glueContext.write_dynamic_frame.from_options(
    frame=cartera_node,
    connection_type="redshift",
    connection_options={
        "postactions": """
        """,
        "redshiftTmpDir": "s3://aws-glue-assets-575238426438-us-east-1/temporary/",
        "useConnectionProperties": "true",
        "dbtable": "cf_processing.cartera_2r",
        "connectionName": "conexion-redshift",
        "preactions": f"""
            DELETE FROM cf_processing.cartera_2r
            where cart_unidad = last_day(to_date('{ano}-{mes}','yyyy-mm'))
            and cart_emp = '{empresa_pais.get(pais)}';
        """
    },
    transformation_ctx="cartera_node_w"
)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…